<a href="https://colab.research.google.com/github/Qvery-mm/NumericMethods_2/blob/main/lab_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Вычислительный практикум
# Задание №5
### Жесткие системы.
## Ковальчуков Александр
### 321 группа
### Вариант №6

# Условия задачи

In [150]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
A = np.array([[-125, 123.3], 
              [123.3, -123]])
y0 = np.array([[1], 
              [1]])
n = A.shape[0]

# Точное решение

In [151]:
eigenvalues = np.linalg.eig(A)[0]
eigenvectors = np.linalg.eig(A)[1]
U = np.linalg.eig(A)[1]
ff = np.linalg.solve(U, y0)
W = ff.T * U

In [152]:
def f(t):
  x = np.exp(t * eigenvalues)
  return np.dot(W, x.T)

def exact(a, b, h):
  ans = []
  m = int((b - a) / h) + 1
  for i in range(m):
    ans.append(f(a + i * h).reshape(1, n))
  return ans

# Метод Эйлера

In [153]:
def euler(a, b, h):
  y = y0
  m = int((b - a) / h)
  E = np.eye(n)
  ans = [y.reshape(1, n)]
  for i in range(m):
    y = np.dot(E + h * A,  y)
    ans.append(y.reshape(1, n))
  return ans  

In [154]:
euler(0, 0.5, 0.1)

[array([[1, 1]]),
 array([[0.83, 1.03]]),
 array([[ 3.1549, -1.4051]]),
 array([[-53.606233,  54.777547]]),
 array([[ 1291.87883401, -1279.95113399]]),
 array([[-30638.40407321,  30392.31383743]])]

# Обратный метод Эйлера

In [155]:
def reverse_euler(a, b, h):
  y = y0
  m = int((b - a) / h)
  E = np.eye(n)
  ans = [y.reshape(1, n)]
  for i in range(m):
    y = np.dot(np.linalg.inv(E - h * A),  y)
    ans.append(y.reshape(1, n))
  return ans  

In [156]:
reverse_euler(0, 0.5, 0.1)

[array([[1, 1]]),
 array([[0.93128545, 0.9385526 ]]),
 array([[0.87054842, 0.87762516]]),
 array([[0.8138996 , 0.82052685]]),
 array([[0.76094199, 0.76713847]]),
 array([[0.71143035, 0.71722366]])]

# Интерполяционный метод Адамса 2 порядка

In [157]:
def adams_2ord(a, b, h):
  y = y0
  m = int((b - a) / h)
  E = np.eye(n)
  ans = [y.reshape(1, n)]
  for i in range(m):
    y = np.dot(np.dot(np.linalg.inv(E - h * A / 2), E + h * A / 2), y)
    ans.append(y.reshape(1, n))
  return ans  

In [158]:
adams_2ord(0, 0.5, 0.1)

[array([[1, 1]]),
 array([[0.92548594, 0.93994696]]),
 array([[0.86941655, 0.87060796]]),
 array([[0.80569496, 0.81726314]]),
 array([[0.75597264, 0.75787072]]),
 array([[0.70133422, 0.71066611]])]

# Метод Эйлера устойчив при $h < 0.008087$

In [159]:
lamb = max(map(abs, eigenvalues))
h = 2 / lamb
h

0.008087210698287415

# Обратный метод Эйлера устойчив при всех $h$, поскольку с.ч $A$ отрицательны и c.ч. $(E - Ah)^{-1}$ по модулю меньше 1

# Метод Адамса 2 порядка устойчив при всех $h$, поскольку $\lambda_i ((E - hA)^{-1}(E + hA)) = \frac{1 + h \lambda_i(A)}{1 - h \lambda_i(A)}$, что по модулю меньше 1, когда c.ч. $A$ отрицательные

In [187]:
def compare(a, b, h):
  k = int((b - a) / h) + 1
  ex = np.array(exact(a, b, h)).reshape(k, 2)
  eu = np.array(euler(a, b, h)).reshape(k, 2)
  re_eu = np.array(reverse_euler(a, b, h)).reshape(k, 2)
  ad = np.array(adams_2ord(a, b, h)).reshape(k, 2)
  return pd.DataFrame( {"Точное y1": ex[:, 0], 
                      "Точное y2": ex[:, 1],
                      "М. Эйлера y1": ex[:, 0] - eu[:, 0],
                      "М. Эйлера y2": ex[:, 1] - eu[:, 1],
                      "Обр. м. Э y1": ex[:, 0] - re_eu[:, 0],
                      "Обр. м. Э y2": ex[:, 1] - re_eu[:, 1],
                      "м. Адамса y1": ex[:, 0] - ad[:, 0],
                      "м. Адамса y2": ex[:, 1] - ad[:, 1],
                       })

In [188]:
compare(0, 0.5, 0.1)

,Точное y1,Точное y2,М. Эйлера y1,М. Эйлера y2,Обр. м. Э y1,Обр. м. Э y2,м. Адамса y1,м. Адамса y2
0,1.000000,1.000000,0.000000,2.220446e-16,0.000000,2.220446e-16,0.000000,2.220446e-16
1,0.928974,0.936539,0.098974,-9.346094e-02,-0.002311,-2.013543e-03,0.003488,-3.407906e-03
2,0.866521,0.873577,-2.288379,2.278677e+00,-0.004027,-4.047756e-03,-0.002895,2.969436e-03
3,0.808267,0.814849,54.414500,-5.396270e+01,-0.005633,-5.678324e-03,0.002572,-2.414608e-03
4,0.753929,0.760068,-1291.124906,1.280711e+03,-0.007013,-7.070584e-03,-0.002044,2.197168e-03
5,0.703243,0.708970,30639.107317,-3.039160e+04,-0.008187,-8.253628e-03,0.001909,-1.696071e-03


In [189]:
compare(0, 0.5, 0.05)

,Точное y1,Точное y2,М. Эйлера y1,М. Эйлера y2,Обр. м. Э y1,Обр. м. Э y2,м. Адамса y1,м. Адамса y2
0,1.000000,1.000000,0.000000e+00,2.220446e-16,0.000000,2.220446e-16,0.000000,2.220446e-16
1,0.961869,0.969702,4.686901e-02,-4.529834e-02,-0.000874,-2.718237e-04,0.002941,-2.910642e-03
2,0.928974,0.936539,-5.247508e-01,5.228141e-01,-0.001123,-1.086415e-03,-0.002113,2.109194e-03
3,0.897204,0.904511,5.978646e+00,-5.927020e+00,-0.001596,-1.605429e-03,0.001539,-1.507612e-03
4,0.866521,0.873577,-6.792744e+01,6.738305e+01,-0.002054,-2.070031e-03,-0.001091,1.106964e-03
5,0.836887,0.843702,7.720361e+02,-7.657948e+02,-0.002480,-2.499932e-03,0.000811,-7.750601e-04
6,0.808267,0.814849,-8.774314e+03,8.703446e+03,-0.002875,-2.898193e-03,-0.000558,5.871030e-04
7,0.780625,0.786982,9.972189e+04,-9.891639e+04,-0.003240,-3.266559e-03,0.000434,-3.918892e-04
8,0.753929,0.760068,-1.133359e+06,1.124205e+06,-0.003577,-3.606606e-03,-0.000278,3.180799e-04
9,0.728145,0.734074,1.288086e+07,-1.277682e+07,-0.003888,-3.919834e-03,0.000239,-1.909017e-04


In [190]:
compare(0, 0.5, 0.001)

,Точное y1,Точное y2,М. Эйлера y1,М. Эйлера y2,Обр. м. Э y1,Обр. м. Э y2,м. Адамса y1,м. Адамса y2
0,1.000000,1.000000,0.000000,2.220446e-16,0.000000,2.220446e-16,0.000000e+00,2.220446e-16
1,0.998415,1.000186,0.000115,-1.136741e-04,-0.000085,8.386411e-05,4.042000e-06,-4.009295e-06
2,0.997026,1.000179,0.000177,-1.742174e-04,-0.000135,1.326250e-04,6.308821e-06,-6.257750e-06
3,0.995790,1.000021,0.000203,-2.002380e-04,-0.000160,1.572802e-04,7.385190e-06,-7.325369e-06
4,0.994674,0.999746,0.000208,-2.045437e-04,-0.000169,1.657619e-04,7.684630e-06,-7.622335e-06
...,...,...,...,...,...,...,...,...
496,0.705204,0.710946,0.000085,8.543065e-05,-0.000085,-8.536167e-05,9.825150e-09,9.905158e-09
497,0.704713,0.710452,0.000085,8.554333e-05,-0.000085,-8.547428e-05,9.838110e-09,9.918223e-09
498,0.704223,0.709958,0.000085,8.565581e-05,-0.000085,-8.558668e-05,9.851047e-09,9.931265e-09
499,0.703733,0.709464,0.000085,8.576808e-05,-0.000085,-8.569889e-05,9.863961e-09,9.944285e-09
